<a href="https://colab.research.google.com/github/mgstockwell/water_well_prediction/blob/main/Groundwater_Load_USGS_Sites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Setup

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

google_project_name = "msd8654-498-dev"
google_dataset_name = "usgs"

Authenticated


### Google upload blob function

In [ ]:
from google.cloud import storage
from google.cloud import bigquery
import urllib.request
import os, datetime

# note: google_project_name  & google_dataset_name at top (global)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the google storage bucket."""

    storage_client = storage.Client(project=google_project_name)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to Storage Bucket {} successfully . {}".format(
            source_file_name, destination_blob_name, datetime.datetime.now()
        )
    )

# Get data from usgs.gov

In [ ]:
from google.cloud import bigquery

bqclient = bigquery.Client()
#SELECT state_postal_abbreviation FROM `bigquery-public-data.census_utility.fips_codes_states` 
# Download a table.
table = bigquery.TableReference.from_string(
    "bigquery-public-data.census_utility.fips_codes_states"
)
rows = bqclient.list_rows(
    table
)
df = rows.to_dataframe()

# continental states and HI/AK al under 60. US Virgin islands, PR, Guam filtered
df = df[df.state_fips_code < '60']
fips_codes_states = df["state_postal_abbreviation"]
print(fips_codes_states.head())

0    AL
1    AK
2    AZ
3    AR
4    CA
Name: state_postal_abbreviation, dtype: object


In [ ]:
import requests
import json
import pandas as pd
from google.cloud import bigquery
import urllib.request
import os

base_url = "https://nwis.waterdata.usgs.gov/nwis/gwlevels?state_cd={state_cd}"
base_url = base_url + "&group_key=NONE&format=sitefile_output&sitefile_output_format=rdb"
base_url = base_url + "&date_format=YYYY-MM-DD&rdb_compression=file&list_of_search_criteria=state_cd"
column_list = ['agency_cd', 'site_no', 'station_nm', 'site_tp_cd', 'lat_va', 'long_va',
    'dec_lat_va', 'dec_long_va', 'coord_meth_cd', 'coord_acy_cd', 'coord_datum_cd', 
    'dec_coord_datum_cd', 'district_cd', 'state_cd', 'county_cd', 'country_cd', 
    'land_net_ds', 'map_nm', 'map_scale_fc', 'alt_va', 'alt_meth_cd', 'alt_acy_va',
    'alt_datum_cd', 'huc_cd', 'basin_cd', 'topo_cd', 'data_types_cd', 'instruments_cd',
    'construction_dt', 'inventory_dt', 'drain_area_va', 'contrib_drain_area_va', 'tz_cd', 
    'local_time_fg', 'reliability_cd', 'gw_file_cd', 'nat_aqfr_cd', 'aqfr_cd', 'aqfr_type_cd',
    'well_depth_va', 'hole_depth_va', 'depth_src_cd', 'project_no', 'rt_bol', 'peak_begin_date',
    'peak_end_date', 'peak_count_nu', 'qw_begin_date', 'qw_end_date', 'qw_count_nu',
    'gw_begin_date', 'gw_end_date', 'gw_count_nu', 'sv_begin_date', 'sv_end_date', 'sv_count_nu']

for c in column_list:
  base_url = base_url +"&column_name=" + c
# print(base_url)

for state_cd in fips_codes_states:
  f = open("groundwater_sites_" + state_cd + ".tmp", 'w')
  f2 = open("groundwater_sites_" + state_cd + ".tsv", 'w')
  url = base_url.replace("{state_cd}",state_cd.lower())
  payload={}
  headers = {}
  response = requests.request("GET", url, headers=headers, data=payload)
  print(state_cd,"Response code:", response.status_code) 
  if (response.status_code>229):
    print("ERROR")
    break;
  else:
    f.writelines(response.text)
    f.close()

  for line in open("groundwater_sites_" + state_cd + ".tmp", 'r'):
    if (line.startswith("#") or line.startswith("5s")):
      #print(line)
      None
    else:
      f2.writelines(line)
  f2.close()

# Combine tsv files, data cleansing

In [ ]:
import os
import pandas as pd

files = os.listdir() 
file_list = list() 

# Create master file and write headers
groundwater_sites = open('groundwater_sites.tsv','w')
with open('groundwater_sites_AK.tsv','r') as f:
  headers = f.readline()
  groundwater_sites.writelines(headers)

for file in os.listdir():
  if file.startswith("groundwater_sites_"):
    with open(file) as f:
      lines=f.readlines()
      # this skips header
      groundwater_sites.writelines(lines[1:])

groundwater_sites.close()

df = pd.read_csv('groundwater_sites.tsv', delimiter='\t')
df.info()
df = df.convert_dtypes()
df.to_csv("groundwater_sites.csv")

bucket_name = google_project_name + "-" + google_dataset_name
upload_blob(bucket_name,'groundwater_sites.csv','groundwater_sites.csv' )


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,4,5,6,7,12,13,14,16,18,19,21,23,24,28,29,30,31,39,40,43,46,49,52,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1821218 entries, 0 to 1821217
Data columns (total 56 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   agency_cd              object
 1   site_no                object
 2   station_nm             object
 3   site_tp_cd             object
 4   lat_va                 object
 5   long_va                object
 6   dec_lat_va             object
 7   dec_long_va            object
 8   coord_meth_cd          object
 9   coord_acy_cd           object
 10  coord_datum_cd         object
 11  dec_coord_datum_cd     object
 12  district_cd            object
 13  state_cd               object
 14  county_cd              object
 15  country_cd             object
 16  land_net_ds            object
 17  map_nm                 object
 18  map_scale_fc           object
 19  alt_va                 object
 20  alt_meth_cd            object
 21  alt_acy_va             object
 22  alt_datum_cd           object
 23  huc_cd 

# Upload file to GCS Bucket

In [ ]:
import os, datetime

# if running locally will need app credentials 
# terminal
# GOOGLE_APPLICATION_CREDENTIALS=<path>/<filename>.json
# powershell:
# $env:GOOGLE_APPLICATION_CREDENTIALS="<path>/<filename>.json"

# List the tsv files and load to BQ
from os import listdir
from os.path import isfile, join
files = [f for f in os.listdir() if (isfile(f))]
bucket_name = google_project_name + "-" + google_dataset_name
for f in files:
 try: 
    upload_blob(bucket_name, f , f)
 except BaseException as err:
    print(f,type(err))
    print(err)

File groundwater_sites_SD.tsv uploaded to Storage Bucket with Bob name  groundwater_sites_SD.tsv successfully . 2022-05-01 18:44:37.010330
File groundwater_sites_WA.tmp uploaded to Storage Bucket with Bob name  groundwater_sites_WA.tmp successfully . 2022-05-01 18:44:37.714901
File groundwater_sites_AL.tsv uploaded to Storage Bucket with Bob name  groundwater_sites_AL.tsv successfully . 2022-05-01 18:44:38.209740
File groundwater_sites_NJ.tsv uploaded to Storage Bucket with Bob name  groundwater_sites_NJ.tsv successfully . 2022-05-01 18:44:38.642378
File groundwater_sites_IA.tmp uploaded to Storage Bucket with Bob name  groundwater_sites_IA.tmp successfully . 2022-05-01 18:44:39.069240
File groundwater_sites_KY.tmp uploaded to Storage Bucket with Bob name  groundwater_sites_KY.tmp successfully . 2022-05-01 18:44:39.514011
File groundwater_sites_TN.tmp uploaded to Storage Bucket with Bob name  groundwater_sites_TN.tmp successfully . 2022-05-01 18:44:39.849160
File groundwater_sites_MS.t

# Upload files to BigQuery Table

In [ ]:
# https://cloud.google.com/bigquery/docs/batch-loading-data#bq
import os, sys, datetime
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project=google_project_name)
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV, 
    fieldDelimiter="\t",
    skip_leading_rows=1, 
    autodetect=True
)

# List the tsv files and load to BQ
from os import listdir
from os.path import isfile, join
files = [f for f in os.listdir() if (isfile(f) and f.endswith(".tsv"))]

for f in files:
  table_name = f.split(".")[0]
  table_id = google_project_name + "." + google_dataset_name + "." + table_name
  with open(f, "rb") as source_file:
    try:
      job = client.load_table_from_file(source_file, table_id, job_config=job_config)

      job.result()  # Waits for the job to complete.

      table = client.get_table(table_id)  # Make an API request.
      print(
          "Loaded {} rows and {} columns to {} at {}".format(
              table.num_rows, len(table.schema), table_id, datetime.datetime.now())
          )
    except BaseException as err:
      print(f, table_name, type(err))
      print(err)


Loaded 64692 rows and 56 columns to msd8654-498-dev.usgs.groundwater_sites_SD at 2022-05-01 20:45:31.926462
Loaded 14787 rows and 56 columns to msd8654-498-dev.usgs.groundwater_sites_AL at 2022-05-01 20:45:35.767831
Loaded 14004 rows and 56 columns to msd8654-498-dev.usgs.groundwater_sites_NJ at 2022-05-01 20:45:41.734046
groundwater_sites.tsv groundwater_sites <class 'google.api_core.exceptions.BadRequest'>
400 Provided Schema does not match Table msd8654-498-dev:usgs.groundwater_sites. Field site_no has changed type from STRING to INTEGER
Loaded 127704 rows and 56 columns to msd8654-498-dev.usgs.groundwater_sites_MS at 2022-05-01 20:46:45.279539
Loaded 4006 rows and 56 columns to msd8654-498-dev.usgs.groundwater_sites_VT at 2022-05-01 20:46:48.633671
Loaded 6212 rows and 56 columns to msd8654-498-dev.usgs.groundwater_sites_TN at 2022-05-01 20:46:54.971237
Loaded 15766 rows and 56 columns to msd8654-498-dev.usgs.groundwater_sites_IA at 2022-05-01 20:47:00.466902
Loaded 55120 rows and 